<h1><center>Exemplo do SCIP </center></h1>


<h2><center>Abertura de Depósitos e Transferência de Produtos </center></h2>


Uma empresa deseja abrir depósitos em quatro cidades: São Carlos, Campinas, Sorocaba e Limeira. Cada depósito pode transportar 100 produtos por semana. O custo fixo para manutenção de cada depósito aberto varia em cada cidade, como mostrado na Tabela 1. Três regiões do Brasil demandam produtos: a Região 1 solicita 80 produtos por semana, a Região 2 solicita 70 produtos por semana, e a Região 3 solicita 40 produtos por semana. Os custos de enviar um produto de um depósito até uma determinada Região são dados na Tabela 2. Deseja-se atender a demanda total semanal de cada região com o menor custo possível, de acordo com as seguintes informações:


1. Se um depósito é aberto em São Carlos, então o depósito de Campinas também precisa ser aberto.
2. No máximo dois depósitos podem ser abertos.
3. O depósito de Sorocaba ou de Limeira precisa ser aberto.





Elabore um modelo matemática de otimização linear para minimizar os custos de distribuição.

<h3><center>Tabela 1 - Custos de abertura dos depósitos </center></h3>

\begin{array}{c|c}
\hline
Cidade & Custo \ Fixo \\ \hline
São \ Carlos &  R$400 \\
Campinas &  R$500 \\
Sorocaba &  R$300 \\
Limeira &  R$150 \\\hline
\end{array}

<h3><center>Tabela 2 - Custos unitários de envio de mercadorias</center></h3>

\begin{array}{c|c|c|c}
\hline
De & Para \ Região \ 1 & Para \ Região \ 2 & Para \ Região \ 3\\ \hline
São \ Carlos &  R$20 &  R$40 &  R$50 \\
Campinas &  R$48 &  R$15 &  R$26 \\
Sorocaba &  R$26 &  R$35 &  R$18 \\
Limeira &  R$24 &  R$50 &  R$35 \\ \hline
\end{array}


<h2><center> Resolução </center></h2>

### Conjuntos
$D$: conjunto de depósitos = {São Carlos, Campinas, Sorocaba, Limeira}.

$R$: conjunto de regiões = {Região 1, Região 2, Região 3}

### Parâmetros

${CF}_{i}$: custo fixo de abertura do depósito $i$.

$T_{ij}$: custo de envio de um produto do depóstio $i$ para a região $j$.

${d}_{j}$: demanda de cada região $j$.


### Variáveis de Decisão
${x}_i \in \{0,1\}$: é igual a 1 se o depósito $i$ é aberto; e igual a 0 caso contrário.

${p}_{ij} \geq 0$: quantidade de produtos transportados do depósito $i$ para a região $j$.

### Função Objetivo

- **Custo total**: Minimiza o custo para entrega da demanda e de abertura de depósitos:

\begin{equation}
\text{Min} \quad Z = \sum_{i \in D}\text{CF}_i \ {x}_{i} + \sum_{i \in D \\ j \in R} T_{ij} \ {p}_{ij}
\tag{0}
\end{equation}

### Restrições

- **Controle de envio máximo permitido para cada depósito**:

\begin{equation}
\sum_{j \in R} {p}_{ij} \leq 100 \ \text{x}_{i} \quad \forall i \in D
\tag{1}
\end{equation}

- **Cumprimento da demanda por região**:

\begin{equation}
\sum_{i \in D} {p}_{ij} \geq {d}_{j} \quad \forall j \in R
\tag{2}
\end{equation}

- **Restrição para Condição 1**:

\begin{equation}
{x}_{Campinas} \geq {x}_{São \ Carlos}
\tag{1}
\end{equation}

- **Restrição para Condição 2**:

\begin{equation}
\sum_{i \in D} {x}_{i} \leq 2
\tag{2}
\end{equation}

- **Restrição para Condição 3**:

\begin{equation}
{x}_{Sorocaba} + {x}_{Limeira} \geq 1
\tag{3}
\end{equation}

- **Domínio das Variáveis**:

\begin{equation}
{x}_{i} \in \{0,1\} \quad \forall i \in D
\tag{4}
\end{equation}

\begin{equation}
{p}_{ij} \geq 0 \quad \forall i \in D, \forall j \in R
\tag{5}
\end{equation}

In [ ]:
# Instalando ambiente anaconda
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# Instalando pacote pyscipopt para uso do SCIP
!conda install pyscipopt


Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.4



# All requested packages already installed.



https://scipbook.readthedocs.io/en/latest/flp.html

In [ ]:
# Importando a classe Modelo e as funções quicksum e multidict do pacote pyscipopt

from pyscipopt import Model, quicksum, multidict


In [ ]:
# Desenvolvendo uma função para implementação do modelo apresentado acima

def building_model(D, R, fixed_costs, transportation_costs, demands):
    model = Model("Depots_Costs")
    x,p = {},{}
    for i in D:
      x[i] = model.addVar(vtype="BINARY", name="x(%s)"%(i))
    for i in D:
      for j in R:
        p[i,j] = model.addVar(vtype="INTEGER", name="p(%s,%s)"%(i,j))
    print(x)
    print(p)

    for i in D:
      model.addCons(quicksum(p[i,j] for j in R) <= 100*x[i], "Controle_Envio_Máximo(%s)"%i)
      # print(i)
    for j in R:
      model.addCons(quicksum(p[i,j] for i in D) >= demands[j], "Cumprimento_Demanda(%s)"%j)
    model.addCons(x['Campinas'] >= x['SaoCarlos'])
    model.addCons(quicksum(x[i] for i in D) <= 2, "Abertura dos Depósitos")
    model.addCons(x['Sorocaba'] + x['Limeira'] >= 1)

    model.setObjective(
        quicksum(fixed_costs[i] * x[i] for i in D) +
        quicksum(transportation_costs[i,j] * p[i,j] for i in D for j in R),
        "minimize")
    model.data = x,p
    return model

In [ ]:
# Criando dicionários para utilização na construção do Modelo

regioes = ['1', '2', '3']
depositos = ['SaoCarlos', 'Campinas', 'Sorocaba', 'Limeira']
demandas = [80, 70, 40]
transporte = [20, 40, 50, 48, 15, 26, 26, 35, 18, 24, 50, 35]
custos_depositos = [400, 500, 300, 150]
depositos, custos_fixos = multidict({i:j for i, j in zip(depositos, custos_depositos)})
temp_transporte = [(i, j) for i in depositos for j in regioes]
temp_transporte, custos_transporte = multidict({i:c for i, c in zip(temp_transporte, transporte)})
demandas, demandas_regioes = multidict({i:j for i, j in zip(regioes, demandas)})


In [ ]:
print(temp_transporte)
print(transporte)
print(demandas)
print(demandas_regioes)

[('SaoCarlos', '1'), ('SaoCarlos', '2'), ('SaoCarlos', '3'), ('Campinas', '1'), ('Campinas', '2'), ('Campinas', '3'), ('Sorocaba', '1'), ('Sorocaba', '2'), ('Sorocaba', '3'), ('Limeira', '1'), ('Limeira', '2'), ('Limeira', '3')]
[20, 40, 50, 48, 15, 26, 26, 35, 18, 24, 50, 35]
['1', '2', '3']
{'1': 80, '2': 70, '3': 40}


In [ ]:
print(depositos)
print(custos_fixos)
print(transporte)
print(custos_transporte)


['SaoCarlos', 'Campinas', 'Sorocaba', 'Limeira']
{'SaoCarlos': 400, 'Campinas': 500, 'Sorocaba': 300, 'Limeira': 150}
[20, 40, 50, 48, 15, 26, 26, 35, 18, 24, 50, 35]
{('SaoCarlos', '1'): 20, ('SaoCarlos', '2'): 40, ('SaoCarlos', '3'): 50, ('Campinas', '1'): 48, ('Campinas', '2'): 15, ('Campinas', '3'): 26, ('Sorocaba', '1'): 26, ('Sorocaba', '2'): 35, ('Sorocaba', '3'): 18, ('Limeira', '1'): 24, ('Limeira', '2'): 50, ('Limeira', '3'): 35}


In [ ]:
# Construindo e solucionando o Modelo

model = building_model(depositos, regioes, custos_fixos, custos_transporte, demandas_regioes)
model.optimize()

{'SaoCarlos': x(SaoCarlos), 'Campinas': x(Campinas), 'Sorocaba': x(Sorocaba), 'Limeira': x(Limeira)}
{('SaoCarlos', '1'): p(SaoCarlos,1), ('SaoCarlos', '2'): p(SaoCarlos,2), ('SaoCarlos', '3'): p(SaoCarlos,3), ('Campinas', '1'): p(Campinas,1), ('Campinas', '2'): p(Campinas,2), ('Campinas', '3'): p(Campinas,3), ('Sorocaba', '1'): p(Sorocaba,1), ('Sorocaba', '2'): p(Sorocaba,2), ('Sorocaba', '3'): p(Sorocaba,3), ('Limeira', '1'): p(Limeira,1), ('Limeira', '2'): p(Limeira,2), ('Limeira', '3'): p(Limeira,3)}


In [ ]:
# Imprimindo o valor da função objetivo e as variáveis com valores maiores que zero

EPS = 1.e-6
x,p = model.data
depositos_abertos = [j for j in x if model.getVal(x[j]) > EPS]
envios_realizados = [(i,j) for (i,j) in p if model.getVal(p[i,j]) > EPS]
print("Valor ótimo encontrado: ", model.getObjVal())
print("Depósitos Abertos: ", depositos_abertos)
print("Envios realizados: ", envios_realizados)

Valor ótimo encontrado:  4750.0
Depósitos Abertos:  ['Campinas', 'Limeira']
Envios realizados:  [('Campinas', '2'), ('Campinas', '3'), ('Limeira', '1'), ('Limeira', '3')]


In [ ]:
# Imprimindo as variáveis e os seus respectivos valores finais

for v in model.getVars():
  print("%s: %d" % (v, round(model.getVal(v))))

x(SaoCarlos): 0
x(Campinas): 1
x(Sorocaba): 0
x(Limeira): 1
p(SaoCarlos,1): 0
p(SaoCarlos,2): 0
p(SaoCarlos,3): 0
p(Campinas,1): 0
p(Campinas,2): 70
p(Campinas,3): 30
p(Sorocaba,1): 0
p(Sorocaba,2): 0
p(Sorocaba,3): 0
p(Limeira,1): 80
p(Limeira,2): 0
p(Limeira,3): 10


In [ ]:
# Escrevendo modelo em arquivo

print(model.writeProblem())

wrote problem to file /content/model.cip
None
